# 1 - Import Packages and Load Audio

In this step, we import the required libraries, such as `librosa` for audio processing, `numpy` for numerical computations, and `matplotlib` for visualization. We load the audio file and inspect its waveform to gain an initial understanding of the speech signal.

In [19]:
# Import necessary libraries
import librosa

# Load the audio signal and sampling rate from the file
signal, fs = librosa.load('record.wav', sr=None)

# 2 - Pre-emphasis

Pre-emphasis is applied to the signal to amplify higher frequencies, which are typically weaker in speech. This step helps to balance the frequency spectrum, improving the signal-to-noise ratio for further analysis.

In [20]:
import numpy as np
import librosa

def pre_emphasis(signal, alpha=0.98):
    """
    Apply pre-emphasis to the input audio signal.

    Args:
        signal (numpy.ndarray): The input audio signal.
        alpha (float): Pre-emphasis filter coefficient. Default is 0.98.

    Returns:
        numpy.ndarray: The emphasized signal.
    """
    emphasized_signal = np.append(signal[0], signal[1:] - alpha * signal[:-1])
    return emphasized_signal

def normalize_signal(signal):
    """
    Normalize the signal to have values between -1 and 1.
    
    Args:
        signal (numpy.ndarray): The input signal.

    Returns:
        numpy.ndarray: The normalized signal.
    """
    return signal / np.max(np.abs(signal))

# Apply pre-emphasis to the original signal
emphasized_signal = pre_emphasis(signal, alpha=0.98)  # 尝试增加 alpha 为 0.98
emphasized_signal = normalize_signal(emphasized_signal)  # 归一化信号

# 3 - Windowing

In this step, the speech signal is divided into short overlapping frames by applying a window function (e.g., Hamming window). This allows for localized analysis of the signal in both time and frequency domains, necessary for short-term spectral processing.

In [21]:
def framing(signal, frame_size, frame_stride, fs):
    """
    Frame the signal into overlapping frames and apply a window function (Hamming window).

    Args:
        signal (numpy.ndarray): The input audio signal.
        frame_size (float): Frame size in seconds.
        frame_stride (float): Frame stride in seconds.
        fs (int): Sampling rate of the signal.

    Returns:
        numpy.ndarray: A 2D array where each row is a frame of the signal.
    """
    # Convert frame size and stride from seconds to samples
    frame_length, frame_step = frame_size * fs, frame_stride * fs
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))

    # Calculate total number of frames and pad the signal if necessary
    signal_length = len(signal)
    total_frames = int(np.ceil(float(np.abs(signal_length - frame_length) / frame_step)))
    padded_signal_length = total_frames * frame_step + frame_length

    # Zero-padding the signal to match the required frame length
    zeros = np.zeros((padded_signal_length - signal_length))
    padded_signal = np.append(signal, zeros)

    # Create indices for frames (each row corresponds to a frame)
    indices = np.tile(np.arange(0, frame_length), (total_frames, 1)) + np.tile(np.arange(0, total_frames * frame_step, frame_step), (frame_length, 1)).T

    # Extract frames from the padded signal
    frames = padded_signal[indices.astype(np.int32, copy=False)]

    # Apply a Hamming window to each frame
    window = np.hamming(frame_length)
    frames_windowed = frames * window

    return frames, frames_windowed, frame_length, total_frames


# Define frame size and stride in seconds
frame_size = 0.025
frame_stride = 0.01

# Apply framing and windowing to the emphasized signal
frames, frames_windowed, frame_length, total_frames = framing(emphasized_signal, frame_size, frame_stride, fs)

# 4 - Short-Time Fourier Transform (STFT)

STFT is used to convert the windowed time-domain signal into the frequency domain. Each frame is transformed into its frequency components, producing a spectrogram that represents how frequencies evolve over time.

In [22]:
def stft(frames, NFFT):
    """
    Perform Short-Time Fourier Transform on the input frames.

    Args:
        frames (numpy.ndarray): The input frames, each row is a frame.
        NFFT (int): Number of FFT points, determines the frequency resolution.

    Returns:
        numpy.ndarray: The magnitude spectrum of each frame.
    """
    # Compute the magnitude of the FFT for each frame
    # rfft computes the one-dimensional n-point discrete Fourier Transform (DFT)
    # and returns the non-negative frequency terms (real FFT).
    mag_frames = np.abs(np.fft.rfft(frames, NFFT))

    # Compute the power spectrum (squared magnitude normalized by the number of FFT points)
    # Power spectrum gives the distribution of power into frequency components.
    pow_frames = ((1.0 / NFFT) * (mag_frames ** 2))

    return pow_frames

# Set the number of FFT points (frequency resolution)
NFFT = 512

# Perform STFT on the frames
spectrum = stft(frames, NFFT)

# 5 - Mel-filter Bank

The Mel-filter bank is applied to the STFT output to approximate human hearing perception. It compresses the frequency scale to focus more on lower frequencies, mimicking how humans perceive sound. This results in the Mel spectrum.

In [23]:
def mel_filter_bank(num_filters, NFFT, fs):
    """
    Generate Mel filter banks.

    Args:
        num_filters (int): The number of Mel filters.
        NFFT (int): Number of FFT points, determines frequency resolution.
        fs (int): Sampling rate of the signal.

    Returns:
        numpy.ndarray: Mel filter banks, shape (num_filters, NFFT // 2 + 1).
    """
    # Convert the low and high frequencies to the Mel scale
    low_freq_mel = 0
    high_freq_mel = 2595 * np.log10(1 + (fs / 2) / 700)

    # Create evenly spaced Mel points
    mel_points = np.linspace(low_freq_mel, high_freq_mel, num_filters + 2)

    # Convert Mel points back to Hz
    hz_points = 700 * (10 ** (mel_points / 2595) - 1)

    # Map Hz points to corresponding FFT bin numbers
    bin_points = np.floor((NFFT + 1) * hz_points / fs)

    # Initialize the filter bank matrix
    filters = np.zeros((num_filters, int(np.floor(NFFT / 2 + 1))))

    # Create triangular filters between successive Mel points
    for m in range(1, num_filters + 1):
        f_m_minus = int(bin_points[m - 1])
        f_m = int(bin_points[m])
        f_m_plus = int(bin_points[m + 1])

        # Construct the left side of the triangular filter
        for k in range(f_m_minus, f_m):
            filters[m - 1, k] = (k - bin_points[m - 1]) / (bin_points[m] - bin_points[m - 1])

        # Construct the right side of the triangular filter
        for k in range(f_m, f_m_plus):
            filters[m - 1, k] = (bin_points[m + 1] - k) / (bin_points[m + 1] - bin_points[m])

    return filters

# Set the number of Mel filters
num_filters = 40

# Generate the Mel filter bank and apply it to the spectrum
filters = mel_filter_bank(num_filters, NFFT, fs)
mel_spectrum = np.dot(spectrum, filters.T)

# Replace zero values in the Mel spectrum with a small positive value to avoid log issues
mel_spectrum = np.where(mel_spectrum == 0, np.finfo(float).eps, mel_spectrum)


# 6 - Log Transformation

After applying the Mel-filter bank, we take the logarithm of the Mel spectrum to convert the amplitudes to a logarithmic scale, which aligns with how the human ear perceives sound intensity changes.

In [24]:
def log_magnitude(x):
    """
    Apply logarithmic compression to the input spectrum to simulate human perception.

    Args:
        x (numpy.ndarray): The input spectrum (e.g., Mel spectrum).

    Returns:
        numpy.ndarray: The logarithmically compressed spectrum.
    """
    # Convert to logarithmic scale (in dB)
    return 10 * np.log10(x)

# Apply log transformation to the Mel spectrum
log_mel_spectrum = log_magnitude(mel_spectrum)



# 7 - Discrete Cosine Transform (DCT)

DCT is applied to the log Mel spectrum to obtain the Mel Frequency Cepstral Coefficients (MFCCs). These coefficients represent the most important characteristics of the speech signal and are often used for tasks like speech recognition.

In [26]:
from scipy.fftpack import dct
# Apply DCT to the log Mel spectrum to compute MFCC features
mfcc_features = dct(log_mel_spectrum, type=2, axis=1, norm='ortho')[:, :13]


# 8 - Dynamic Feature Extraction

In this step, the first-order (Delta) and second-order (Delta-Delta) derivatives of MFCCs are calculated. These dynamic features capture changes in the speech signal over time, providing additional temporal information.

In [27]:
def delta(feature_matrix, N=2):
    """
    Calculate delta (derivative) of the feature matrix.

    Args:
        feature_matrix (numpy.ndarray): Input feature matrix (e.g., MFCCs).
        N (int): The window size for calculating the delta.

    Returns:
        numpy.ndarray: Delta feature matrix.
    """
    # Number of frames in the feature matrix
    num_frames, _ = feature_matrix.shape

    # Denominator for the delta calculation
    denominator = 2 * sum([i ** 2 for i in range(1, N + 1)])

    # Initialize the delta feature matrix with the same shape
    delta_feature = np.empty_like(feature_matrix)

    # Pad the feature matrix at the edges to handle boundary conditions
    padded = np.pad(feature_matrix, ((N, N), (0, 0)), mode='edge')

    # Compute the delta for each frame
    for t in range(num_frames):
        delta_feature[t] = np.dot(np.arange(-N, N + 1), padded[t: t + 2 * N + 1]) / denominator

    return delta_feature

# Compute the first-order delta (Delta) of the MFCC features
delta1 = delta(mfcc_features)

# Compute the second-order delta (Delta-Delta) of the first-order delta
delta2 = delta(delta1)

# 9 - Feature Transformation

The MFCCs, along with their dynamic features (Delta and Delta-Delta), are normalized to ensure that each feature has a mean of zero and unit variance. This helps to remove biases and scale differences between features, improving model performance.

In [28]:
# Stack the MFCC, Delta, and Delta-Delta features horizontally (combine them into one feature set)
stacked_features = np.hstack((mfcc_features, delta1, delta2))

# Mean normalization: subtract the mean of each feature across all frames
cmn_features = stacked_features - np.mean(stacked_features, axis=0)

# Variance normalization: divide by the standard deviation of each feature
cvn_features = cmn_features / np.std(cmn_features, axis=0)


# 10 - Principal Component Analysis (PCA)

PCA is performed to reduce the dimensionality of the feature set while retaining the most important information. By keeping only the most significant principal components, this step helps to remove redundancy and improve computational efficiency.

In [29]:
from sklearn.decomposition import PCA
import pickle

def feature_transformation(features, n_components=12):
    """
    Perform Principal Component Analysis (PCA) on the feature set.

    Args:
        features (numpy.ndarray): The input features to be transformed.
        n_components (int): Number of principal components to keep.

    Returns:
        tuple: (Transformed features, PCA object)
    """
    # Initialize PCA with the desired number of components
    pca = PCA(n_components=n_components)

    # Fit the PCA model to the features and transform the data
    transformed_features = pca.fit_transform(features)

    return transformed_features, pca

# Apply PCA to reduce the dimensionality of the stacked features (MFCC, Delta, Delta-Delta)
transformed_features, pca_model = feature_transformation(stacked_features)

# 保存特征值到 pkl 文件
with open('features.pkl', 'wb') as f:
    pickle.dump(transformed_features, f)
    pickle.dump(pca_model, f)


# 11 - Comparison with `librosa` MFCC

In this step, we compare the custom MFCC implementation with the MFCCs generated by the librosa library to validate our custom implementation.

In [30]:
# Calculate the hop length based on the frame stride and sampling rate
hop_length = int(frame_stride * fs)

# Use librosa to compute MFCCs directly from the signal
librosa_mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=13, n_fft=NFFT, hop_length=hop_length, n_mels=num_filters)

# Ensure the custom MFCCs and librosa MFCCs have the same shape by trimming or padding as necessary
if librosa_mfcc.shape[1] > mfcc_features.shape[0]:
    # Trim the librosa MFCC to match the custom MFCC length
    librosa_mfcc = librosa_mfcc[:, :mfcc_features.shape[0]]
elif librosa_mfcc.shape[1] < mfcc_features.shape[0]:
    # Trim the custom MFCC to match the librosa MFCC length
    mfcc_features = mfcc_features[:librosa_mfcc.shape[1], :]
